In [1]:
# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%matplotlib inline
import pandas as pd
pd.__version__  # for the record

'1.0.3'

In [3]:
df = pd.read_csv('../data/SPY_20110701_20120630_Bollinger.csv',index_col=0,parse_dates=True)
#df = df.loc['2012-01-01':,:]
df.shape
df.head(3)
df.tail(3)

(252, 9)

,Open,High,Low,Close,Adj Close,Volume,UpperB,LowerB,PercentB
Date,,,,,,,,,
2011-07-01,132.089996,134.100006,131.779999,133.919998,117.161659,202385700,132.373927,125.316073,1.219057
2011-07-05,133.779999,134.080002,133.389999,133.809998,117.065437,165936000,133.254297,124.912703,1.066618
2011-07-06,133.490005,134.139999,133.110001,133.970001,117.205429,143331600,134.040915,124.627085,0.992467


,Open,High,Low,Close,Adj Close,Volume,UpperB,LowerB,PercentB
Date,,,,,,,,,
2012-06-27,132.419998,133.429993,131.970001,133.169998,118.980804,108088000,136.447962,128.140042,0.605441
2012-06-28,132.289993,132.990005,131.279999,132.789993,118.641281,169242100,136.500761,128.219241,0.551922
2012-06-29,135.199997,136.270004,134.850006,136.100006,121.598610,212250900,136.721010,128.792993,0.921670


In [4]:
%matplotlib qt
import matplotlib.pyplot as plt

In [5]:
# def plotbars(v,aa=None,w=0.5,e=0.5):
#     x = range(len(v))
#     fig, ax = plt.subplots()
#     ax.bar(x,v,antialiased=aa,width=w,linewidth=0.0)
#     plt.show(block=False)

In [18]:
def plotbars(v,fid=1,dpi=100,**kwargs):
    x = range(len(v))
    fig = plt.figure(fid,dpi=dpi)
    ax  = fig.add_subplot()
    ax.bar(x,v,**kwargs)
    plt.show(block=False)

In [19]:
v1 = [202385700, 165936000, 143331600, 170464200, 194100500, 195918600, 214675700,
      204062600, 226111800, 220012800, 196872100, 166554900, 137145400, 245246300,
      126019400, 136653800, 131278200, 249020100, 207939900, 307038400, 325790900,
      346653800, 370830800, 520721800, 655619200, 702263900, 717828700, 662607400,
      487979700, 313731600, 258810600, 294095200, 238201100, 512956300, 428281300,
      275090600, 331136600, 246869700, 312365400, 314495900, 190977200, 241315700,
      301828400, 254585900, 255517200, 285130500, 209803200, 250568200, 380195100,
      305793500]

v2 = v1 + v1 + v1

print('len(v1)=',len(v1))
print('len(v2)=',len(v2))


# plotbars(v1,w=0.2,e=0)
# plotbars(v2,w=0.2,e=0)

# plotbars(v1,w=.98,e=0)
# plotbars(v2,w=.98,e=0)

# plotbars(v1,w=1.0,e=0)
# plotbars(v2,w=1.0,e=0)


len(v1)= 50
len(v2)= 150


In [20]:
STOP HERE

SyntaxError: invalid syntax (<ipython-input-20-a6774c8535dd>, line 1)

In [21]:
plotbars(v1)

In [10]:
plotbars(v2)

In [45]:
def adjust_brightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])

In [65]:
c1 = 'c'
c2 = adjust_brightness(c1,0.75)
print(c2)
print(c1)

(0.0, 0.5624999999999998, 0.5625)
c


In [70]:
plotbars(v2,1,dpi=135,ec=c2,fc=c1,linewidth=0.8,width=0.8)
plotbars(v2,2,dpi=135,ec=c2,fc=c1,linewidth=0.5,width=0.5)
#plotbars(v2,2,dpi=150,ec='k',linewidth=0.1)

In [ ]:
v3 = [ (v,float('nan')) for v in v2]

In [ ]:
v3 = [v for tup in v3 for v in tup]

In [ ]:
len(v3)

In [ ]:
plotbars(v2,w=0.5)

In [ ]:

plotbars(v3,w=1.0)

In [ ]:
STOP HERE

In [ ]:
v = df['Volume'].values
#v = df['Close'].values
def plot_vol(v,aa=None,w=0.5):
    fig = plt.figure(figsize=(20,12))
    ax = fig.add_axes( [0.15, 0.18, 0.70, 0.70] )
    x = range(len(v))
    ax.bar(x,v,antialiased=aa,width=w)
    #ax.plot(x,v,antialiased=aa)
    plt.show()

In [ ]:
y = v[0:51]
plot_vol(v,w=0.7)
plot_vol(y,w=0.7)

In [ ]:
print(y)

In [ ]:
STOP HERE

---

#### Using this dataframe, we can of course plot a basic ohlc or candlestick plot:



In [ ]:
import mplfinance as mpf

mpf.plot(df,volume=True)

---

Let's say we want to plot the Lower Bollinger band along with the basic OHLCV plot.  

We use `make_addplot()` to create the addplot dict, and pass that into the plot() function:

In [ ]:
apdict = mpf.make_addplot(df['LowerB'])

mpf.plot(df,volume=True,addplot=apdict)

---

When creating the `addplot` dict, we can specify that we want a scatter plot:

In [ ]:
apd = mpf.make_addplot(df['LowerB'],type='scatter')

mpf.plot(df,addplot=apd)

---

The above example is a trivial use of a scatter plot, where the default line plot makes more sense.  

A more helpful use of a scatter plot might be to highlight specific movements in the data.  For example, let's say we want to highlight whenever the "Percent B" Bollinger metric drops below zero.  To do this, let's first calculate a series that contains this information:

In [ ]:
def percentB_belowzero(percentB,price):
    import numpy as np
    signal   = []
    previous = -1.0
    for date,value in percentB.iteritems():
        if value < 0 and previous >= 0:
            signal.append(price[date]*0.99)
        else:
            signal.append(np.nan)
        previous = value
    return signal

---
Take a small data set, and calculate a series that shows when the percentB falls below zero:

In [ ]:
tdf = df.loc['05-10-2012':'06-07-2012',]  # Take a smaller data set so it's easier to see the scatter points

signal = percentB_belowzero(tdf['PercentB'], tdf['Close'])

---

Now plot the calculated information as an additional scatter plot on top of the the OHLC data:

In [ ]:
apd = mpf.make_addplot(signal,type='scatter')

mpf.plot(tdf,addplot=apd)

---

We can customize the marker size and shape, to make the scatter markers easier to see:

In [ ]:
apd = mpf.make_addplot(signal,type='scatter',markersize=200,marker='^')

mpf.plot(tdf,addplot=apd)

---

## Plotting multiple additional data sets

There are two ways to plot multiple additional data sets.

- If the configuration is the same for all additional data sets, simply pass a `DataFrame` for the data.  All columns in the DataFrame will be plotted.

- Alternatively you can create multiple `dict`s and pass a `list` of `dict`s to the `addplot` keyword


---

Passing a DataFrame as the addplot data plots all columns in the DataFrame:

In [ ]:
tcdf = df[['LowerB','UpperB']]  # DataFrame with two columns
apd  = mpf.make_addplot(tcdf)
mpf.plot(df,addplot=apd)

---

Setting `addplot=` a `list` of `dict`s is another to create multiple additional plots.<br>This method is necessary if the additional plots will have different configurations.  For example:

---

First prepare the data:

In [ ]:
def percentB_aboveone(percentB,price):
    import numpy as np
    signal   = []
    previous = 2
    for date,value in percentB.iteritems():
        if value > 1 and previous <= 1:
            signal.append(price[date]*1.01)
        else:
            signal.append(np.nan)
        previous = value
    return signal

In [ ]:
low_signal  = percentB_belowzero(df['PercentB'], df['Close']) 
high_signal = percentB_aboveone(df['PercentB'], df['Close'])

---

Now create the additional plot `dict`s and plot the data: 

In [ ]:
apds = [ mpf.make_addplot(tcdf),
         mpf.make_addplot(low_signal,type='scatter',markersize=200,marker='^'),
         mpf.make_addplot(high_signal,type='scatter',markersize=200,marker='v'),
       ]

mpf.plot(df,addplot=apds,figscale=1.25,volume=True)

---

## Plotting additional data on panel "B"

---
We refer to the Main, Upper panel as Panel "A" and the Lower panel as Panel "B".

It is possible to plot the additional data on Panel "B" (where volume is usually plotted).
In this example, as is typical in Bollinger Band Analysis, we plot `PercentB` in panel B:

In [ ]:
apds = [ mpf.make_addplot(tcdf),
         mpf.make_addplot(low_signal,type='scatter',markersize=200,marker='^'),
         mpf.make_addplot(high_signal,type='scatter',markersize=200,marker='v'),
         mpf.make_addplot((df['PercentB']),panel=1,color='g')
       ]

mpf.plot(df,addplot=apds,figscale=1.3,volume=True)

---

## Plotting additional data with a *secondary y-axis*

---

- Notice in the above plot, in the lower panel with the "Volume" bars, we see that "Percent B" line ***has its own y-axis on the right side.***


- `mpf.make_addplot()` has a keyword argument called `secondary_y` which can have **three** possible values: **`True`**, **`False`**, and **`'auto'`**.
  - The default value is `'auto'` which means if you don't specify `secondary_y`, or if you specify `secondary_y='auto'`, then `mpf.plot()` will attempt to decide whether a secondary y-axis is needed, by comparing the order of magnitude of the addplot data with the order of magnitude of the data that is already on the plot.
  - If **`mpf.plot()`** gets it wrong, you can always override by setting **`secondary_y=True`** or **`secondary_y=False`**.
  
---

- Below we see that `make_addplot()` also allows setting the `linestyle` for each additional plot.


- **Notice also** that we pass an alternative "mplfinance `style`" to demonstrate that **if the `style` specifies that the *primary y-axis* should be on the right, then `mpf.plot()` knows to put any `secondary_y` axes on the left.**

In [ ]:
apds = [ mpf.make_addplot(tcdf,linestyle='dashdot'),
         mpf.make_addplot(low_signal,type='scatter',markersize=200,marker='^'),
         mpf.make_addplot(high_signal,type='scatter',markersize=200,marker='v'),
         mpf.make_addplot((df['PercentB']),panel=1,color='g',linestyle='dotted')
       ]

mpf.plot(df,addplot=apds,figscale=1.5,volume=True,style='starsandstripes')

---
---

* **Below** we demonstrate that the main (upper) panel can also have a `secondary_y` axis:

In [ ]:
import math
# Change order of magnitude and range of low_signal, so that it will require a `secondary_y`:
# note: this calculation has no financial meaning whatsoever; we are just generating some 
#       data to modify the order of magnitude and range, so as to be able to demonstrate 
#       secondary_y on the main panel.
new_low_signal = [x*20.*math.sin(x) for x in low_signal] 

In [ ]:
apds = [mpf.make_addplot(tcdf,linestyle='dashdot'),
        mpf.make_addplot(new_low_signal,type='scatter',markersize=200,marker='^',secondary_y='auto'),
        mpf.make_addplot(high_signal,type='scatter',markersize=200,marker='v',color='orange'),
        mpf.make_addplot((df['PercentB']),panel=1,color='g',linestyle='dotted')
       ]

mpf.plot(df,addplot=apds,figscale=1.5,volume=True,style='sas')

In [ ]:
# The same plot, with a style that puts the primary y-axis on the left:
mpf.plot(df,addplot=apds,figscale=1.5,volume=True,style='default')